<a href="https://colab.research.google.com/github/Ashara-kosi/ML-projects/blob/main/Customer_Response.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
import pandas as pd
import numpy as np
import string 
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS
import spacy
import nltk
!pip install tweet-preprocessor
import preprocessor as p

In [32]:
tweets = pd.read_csv('twcs.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [33]:
tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2695394 entries, 0 to 2695393
Data columns (total 7 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   tweet_id                 int64  
 1   author_id                object 
 2   inbound                  object 
 3   created_at               object 
 4   text                     object 
 5   response_tweet_id        object 
 6   in_response_to_tweet_id  float64
dtypes: float64(1), int64(1), object(5)
memory usage: 143.9+ MB


In [34]:
tweets.isna().sum()

tweet_id                        0
author_id                       1
inbound                         1
created_at                      1
text                            1
response_tweet_id          996624
in_response_to_tweet_id    759507
dtype: int64

In [35]:
#removing null values
tweets = tweets.dropna()

In [36]:
tweets.isna().sum()

tweet_id                   0
author_id                  0
inbound                    0
created_at                 0
text                       0
response_tweet_id          0
in_response_to_tweet_id    0
dtype: int64

In [37]:
tweets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 939264 entries, 0 to 2695392
Data columns (total 7 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   tweet_id                 939264 non-null  int64  
 1   author_id                939264 non-null  object 
 2   inbound                  939264 non-null  object 
 3   created_at               939264 non-null  object 
 4   text                     939264 non-null  object 
 5   response_tweet_id        939264 non-null  object 
 6   in_response_to_tweet_id  939264 non-null  float64
dtypes: float64(1), int64(1), object(5)
memory usage: 57.3+ MB


In [38]:
#using the preprocessing libray to clean text
def preprocess_tweet(row):
    text = row['text']
    text = p.clean(text)
    return text

tweets['clean_tweet'] = tweets.apply(preprocess_tweet,axis=1)

In [39]:
tweets.head(10)

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id,clean_tweet
0,1,sprintcare,False,Tue Oct 31 22:10:47 +0000 2017,@115712 I understand. I would like to assist y...,2,3.0,I understand. I would like to assist you. We w...
2,3,115712,True,Tue Oct 31 22:08:27 +0000 2017,@sprintcare I have sent several private messag...,1,4.0,I have sent several private messages and no on...
3,4,sprintcare,False,Tue Oct 31 21:54:49 +0000 2017,@115712 Please send us a Private Message so th...,3,5.0,Please send us a Private Message so that we ca...
4,5,115712,True,Tue Oct 31 21:49:35 +0000 2017,@sprintcare I did.,4,6.0,I did.
5,6,sprintcare,False,Tue Oct 31 21:46:24 +0000 2017,@115712 Can you please send us a private messa...,"5,7",8.0,"Can you please send us a private message, so t..."
8,12,115713,True,Tue Oct 31 22:04:47 +0000 2017,@sprintcare You gonna magically change your co...,"11,13,14",15.0,You gonna magically change your connectivity f...
9,15,sprintcare,False,Tue Oct 31 20:03:31 +0000 2017,@115713 We understand your concerns and we'd l...,12,16.0,We understand your concerns and we'd like for ...
10,16,115713,True,Tue Oct 31 20:00:43 +0000 2017,@sprintcare Since I signed up with you....Sinc...,15,17.0,Since I signed up with you....Since day
11,17,sprintcare,False,Tue Oct 31 19:59:13 +0000 2017,@115713 H there! We'd definitely like to work ...,16,18.0,H there! We'd definitely like to work with you...
15,21,Ask_Spectrum,False,Tue Oct 31 22:14:37 +0000 2017,@115716 What information is incorrect? ^JK,"22,23",24.0,What information is incorrect? ^JK


In [40]:
#first step of preprocessing text data
#lowercasing
tweets['new_text'] = tweets['clean_tweet'].str.lower()

In [41]:
tweets.head()

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id,clean_tweet,new_text
0,1,sprintcare,False,Tue Oct 31 22:10:47 +0000 2017,@115712 I understand. I would like to assist y...,2,3.0,I understand. I would like to assist you. We w...,i understand. i would like to assist you. we w...
2,3,115712,True,Tue Oct 31 22:08:27 +0000 2017,@sprintcare I have sent several private messag...,1,4.0,I have sent several private messages and no on...,i have sent several private messages and no on...
3,4,sprintcare,False,Tue Oct 31 21:54:49 +0000 2017,@115712 Please send us a Private Message so th...,3,5.0,Please send us a Private Message so that we ca...,please send us a private message so that we ca...
4,5,115712,True,Tue Oct 31 21:49:35 +0000 2017,@sprintcare I did.,4,6.0,I did.,i did.
5,6,sprintcare,False,Tue Oct 31 21:46:24 +0000 2017,@115712 Can you please send us a private messa...,"5,7",8.0,"Can you please send us a private message, so t...","can you please send us a private message, so t..."


In [42]:
#removing punctuations


In [43]:
tweets['new_text'] = tweets['new_text'].str.replace('[^\w\s]','')
tweets.head(10)

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id,clean_tweet,new_text
0,1,sprintcare,False,Tue Oct 31 22:10:47 +0000 2017,@115712 I understand. I would like to assist y...,2,3.0,I understand. I would like to assist you. We w...,i understand i would like to assist you we wou...
2,3,115712,True,Tue Oct 31 22:08:27 +0000 2017,@sprintcare I have sent several private messag...,1,4.0,I have sent several private messages and no on...,i have sent several private messages and no on...
3,4,sprintcare,False,Tue Oct 31 21:54:49 +0000 2017,@115712 Please send us a Private Message so th...,3,5.0,Please send us a Private Message so that we ca...,please send us a private message so that we ca...
4,5,115712,True,Tue Oct 31 21:49:35 +0000 2017,@sprintcare I did.,4,6.0,I did.,i did
5,6,sprintcare,False,Tue Oct 31 21:46:24 +0000 2017,@115712 Can you please send us a private messa...,"5,7",8.0,"Can you please send us a private message, so t...",can you please send us a private message so th...
8,12,115713,True,Tue Oct 31 22:04:47 +0000 2017,@sprintcare You gonna magically change your co...,"11,13,14",15.0,You gonna magically change your connectivity f...,you gonna magically change your connectivity f...
9,15,sprintcare,False,Tue Oct 31 20:03:31 +0000 2017,@115713 We understand your concerns and we'd l...,12,16.0,We understand your concerns and we'd like for ...,we understand your concerns and wed like for y...
10,16,115713,True,Tue Oct 31 20:00:43 +0000 2017,@sprintcare Since I signed up with you....Sinc...,15,17.0,Since I signed up with you....Since day,since i signed up with yousince day
11,17,sprintcare,False,Tue Oct 31 19:59:13 +0000 2017,@115713 H there! We'd definitely like to work ...,16,18.0,H there! We'd definitely like to work with you...,h there wed definitely like to work with you o...
15,21,Ask_Spectrum,False,Tue Oct 31 22:14:37 +0000 2017,@115716 What information is incorrect? ^JK,"22,23",24.0,What information is incorrect? ^JK,what information is incorrect jk


In [44]:

nltk.download('stopwords')
stop = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [45]:
tweets['new_text'] = tweets['new_text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

In [46]:
tweets.head(10)

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id,clean_tweet,new_text
0,1,sprintcare,False,Tue Oct 31 22:10:47 +0000 2017,@115712 I understand. I would like to assist y...,2,3.0,I understand. I would like to assist you. We w...,understand would like assist would need get pr...
2,3,115712,True,Tue Oct 31 22:08:27 +0000 2017,@sprintcare I have sent several private messag...,1,4.0,I have sent several private messages and no on...,sent several private messages one responding u...
3,4,sprintcare,False,Tue Oct 31 21:54:49 +0000 2017,@115712 Please send us a Private Message so th...,3,5.0,Please send us a Private Message so that we ca...,please send us private message assist click me...
4,5,115712,True,Tue Oct 31 21:49:35 +0000 2017,@sprintcare I did.,4,6.0,I did.,
5,6,sprintcare,False,Tue Oct 31 21:46:24 +0000 2017,@115712 Can you please send us a private messa...,"5,7",8.0,"Can you please send us a private message, so t...",please send us private message gain details ac...
8,12,115713,True,Tue Oct 31 22:04:47 +0000 2017,@sprintcare You gonna magically change your co...,"11,13,14",15.0,You gonna magically change your connectivity f...,gonna magically change connectivity whole family
9,15,sprintcare,False,Tue Oct 31 20:03:31 +0000 2017,@115713 We understand your concerns and we'd l...,12,16.0,We understand your concerns and we'd like for ...,understand concerns wed like please send us di...
10,16,115713,True,Tue Oct 31 20:00:43 +0000 2017,@sprintcare Since I signed up with you....Sinc...,15,17.0,Since I signed up with you....Since day,since signed yousince day
11,17,sprintcare,False,Tue Oct 31 19:59:13 +0000 2017,@115713 H there! We'd definitely like to work ...,16,18.0,H there! We'd definitely like to work with you...,h wed definitely like work long experiencing i...
15,21,Ask_Spectrum,False,Tue Oct 31 22:14:37 +0000 2017,@115716 What information is incorrect? ^JK,"22,23",24.0,What information is incorrect? ^JK,information incorrect jk


In [47]:
#using spacy to lemmanize
nlp = spacy.load('en', disable=['parser', 'ner'])
def space(comment):
    doc = nlp(comment)
    return " ".join([token.lemma_ for token in doc])
tweets['new_text']= tweets['new_text'].apply(space)

In [48]:
tweets.head(10)

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id,clean_tweet,new_text
0,1,sprintcare,False,Tue Oct 31 22:10:47 +0000 2017,@115712 I understand. I would like to assist y...,2,3.0,I understand. I would like to assist you. We w...,understand would like assist would need get pr...
2,3,115712,True,Tue Oct 31 22:08:27 +0000 2017,@sprintcare I have sent several private messag...,1,4.0,I have sent several private messages and no on...,send several private message one respond usual
3,4,sprintcare,False,Tue Oct 31 21:54:49 +0000 2017,@115712 Please send us a Private Message so th...,3,5.0,Please send us a Private Message so that we ca...,please send -PRON- private message assist clic...
4,5,115712,True,Tue Oct 31 21:49:35 +0000 2017,@sprintcare I did.,4,6.0,I did.,
5,6,sprintcare,False,Tue Oct 31 21:46:24 +0000 2017,@115712 Can you please send us a private messa...,"5,7",8.0,"Can you please send us a private message, so t...",please send -PRON- private message gain detail...
8,12,115713,True,Tue Oct 31 22:04:47 +0000 2017,@sprintcare You gonna magically change your co...,"11,13,14",15.0,You gonna magically change your connectivity f...,go to magically change connectivity whole family
9,15,sprintcare,False,Tue Oct 31 20:03:31 +0000 2017,@115713 We understand your concerns and we'd l...,12,16.0,We understand your concerns and we'd like for ...,understand concern -PRON- would like please se...
10,16,115713,True,Tue Oct 31 20:00:43 +0000 2017,@sprintcare Since I signed up with you....Sinc...,15,17.0,Since I signed up with you....Since day,since sign yousince day
11,17,sprintcare,False,Tue Oct 31 19:59:13 +0000 2017,@115713 H there! We'd definitely like to work ...,16,18.0,H there! We'd definitely like to work with you...,h -PRON- would definitely like work long exper...
15,21,Ask_Spectrum,False,Tue Oct 31 22:14:37 +0000 2017,@115716 What information is incorrect? ^JK,"22,23",24.0,What information is incorrect? ^JK,information incorrect jk


In [49]:
#removal of stopwords

tweets['new_text']

0          understand would like assist would need get pr...
2             send several private message one respond usual
3          please send -PRON- private message assist clic...
4                                                           
5          please send -PRON- private message gain detail...
                                 ...                        
2695382    please provide -PRON- book number assist accor...
2695385    appreciate could share booking number concern ...
2695386    book number sgjtnl main concern next available...
2695390    hi upon check -PRON- have utilize free move fl...
2695392                                                     
Name: new_text, Length: 939264, dtype: object

In [50]:
nltk.download('punkt')
#finding out the most frequently used words
line = tweets['new_text'].str.cat(sep=' ')
print(line)
words = nltk.tokenize.word_tokenize(line)
word_dist = nltk.FreqDist(words)
dff = pd.DataFrame(word_dist.most_common(), 
                    columns=['Word', 'Frequency'])
dff['Word_Count'] = dff.Word.apply(len)
dff.head(20)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



,Word,Frequency,Word_Count
0,-PRON-,282658,6
1,not,128728,3
2,please,113660,6
3,be,104026,2
4,get,95822,3
5,thank,95740,5
6,help,82478,4
7,would,75591,5
8,hi,75587,2
9,sorry,74465,5


In [51]:
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.sentiment.util import *
#Sentiment Analysis
SIA = SentimentIntensityAnalyzer()
tweets["new_text"]= tweets["new_text"].astype(str)
# Applying Model, Variable Creation
tweets['Polarity Score']=tweets["new_text"].apply(lambda x:SIA.polarity_scores(x)['compound'])
tweets['Neutral Score']=tweets["new_text"].apply(lambda x:SIA.polarity_scores(x)['neu'])
tweets['Negative Score']=tweets["new_text"].apply(lambda x:SIA.polarity_scores(x)['neg'])
tweets['Positive Score']=tweets["new_text"].apply(lambda x:SIA.polarity_scores(x)['pos'])
# Converting 0 to 1 Decimal Score to a Categorical Variable
tweets['Sentiment']=''
tweets.loc[tweets['Polarity Score']>0,'Sentiment']='Positive'
tweets.loc[tweets['Polarity Score']==0,'Sentiment']='Neutral'
tweets.loc[tweets['Polarity Score']<0,'Sentiment']='Negative'

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [52]:
tweets.head(20)

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id,clean_tweet,new_text,Polarity Score,Neutral Score,Negative Score,Positive Score,Sentiment
0,1,sprintcare,False,Tue Oct 31 22:10:47 +0000 2017,@115712 I understand. I would like to assist y...,2,3.0,I understand. I would like to assist you. We w...,understand would like assist would need get pr...,0.5994,0.647,0.000,0.353,Positive
2,3,115712,True,Tue Oct 31 22:08:27 +0000 2017,@sprintcare I have sent several private messag...,1,4.0,I have sent several private messages and no on...,send several private message one respond usual,0.0000,1.000,0.000,0.000,Neutral
3,4,sprintcare,False,Tue Oct 31 21:54:49 +0000 2017,@115712 Please send us a Private Message so th...,3,5.0,Please send us a Private Message so that we ca...,please send -PRON- private message assist clic...,0.4767,0.661,0.000,0.339,Positive
4,5,115712,True,Tue Oct 31 21:49:35 +0000 2017,@sprintcare I did.,4,6.0,I did.,,0.0000,0.000,0.000,0.000,Neutral
5,6,sprintcare,False,Tue Oct 31 21:46:24 +0000 2017,@115712 Can you please send us a private messa...,"5,7",8.0,"Can you please send us a private message, so t...",please send -PRON- private message gain detail...,0.6908,0.513,0.000,0.487,Positive
8,12,115713,True,Tue Oct 31 22:04:47 +0000 2017,@sprintcare You gonna magically change your co...,"11,13,14",15.0,You gonna magically change your connectivity f...,go to magically change connectivity whole family,0.0000,1.000,0.000,0.000,Neutral
9,15,sprintcare,False,Tue Oct 31 20:03:31 +0000 2017,@115713 We understand your concerns and we'd l...,12,16.0,We understand your concerns and we'd like for ...,understand concern -PRON- would like please se...,0.5859,0.676,0.000,0.324,Positive
10,16,115713,True,Tue Oct 31 20:00:43 +0000 2017,@sprintcare Since I signed up with you....Sinc...,15,17.0,Since I signed up with you....Since day,since sign yousince day,0.0000,1.000,0.000,0.000,Neutral
11,17,sprintcare,False,Tue Oct 31 19:59:13 +0000 2017,@115713 H there! We'd definitely like to work ...,16,18.0,H there! We'd definitely like to work with you...,h -PRON- would definitely like work long exper...,0.6369,0.574,0.000,0.426,Positive
15,21,Ask_Spectrum,False,Tue Oct 31 22:14:37 +0000 2017,@115716 What information is incorrect? ^JK,"22,23",24.0,What information is incorrect? ^JK,information incorrect jk,0.2263,0.513,0.000,0.487,Positive
